In [1]:
import os
import xarray as xr
import rioxarray
import geopandas as gpd
from shapely.geometry import mapping
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Inspecting single datafile
data=xr.open_dataset(r"E:\Python\GeoPandas\2018\07\sm2018_07-dk1.v2.3.0.nc")
data


In [ ]:
print(data.dims)

In [ ]:
# Base directory where yearly directories are stored
base_dir = r"E:\Python\GeoPandas\Tamstat_data"

# Load Kenya shapefile
kenya = gpd.read_file(r'E:\Python\GeoPandas\gadm41_KEN_shp\gadm41_KEN_3.shp')

# Create a list to store paths for each year
year_paths = []

# Iterate over each year directory and collect paths and store them in year_paths
for year_dir in os.listdir(base_dir):
    year_path = os.path.join(base_dir, year_dir)
    
    if os.path.isdir(year_path):
        year_paths.append(year_path)

# Prompt user to select a year
print("Available years:")
for idx, year_path in enumerate(year_paths):
    print(f"{idx + 1}. {os.path.basename(year_path)}")

# Input validation and selection
while True:
    try:
        choice = int(input("Enter the number of the year you want to process: "))
        if 1 <= choice <= len(year_paths):
            selected_year_path = year_paths[choice - 1]
            selected_year = os.path.basename(selected_year_path)
            break
        else:
            print("Invalid choice. Please enter a valid number.")
    except ValueError:
        print("Invalid input. Please enter a number.")

# PDF file to save plots
pdf_path = f"Kenya_Average_Soil_Moisture_{selected_year}.pdf"
with PdfPages(pdf_path) as pdf:
    
    # Initialize list to store monthly averages for the selected year
    monthly_averages_selected_year = []

    # Iterate over each month directory within the selected year
    for month_dir in os.listdir(selected_year_path):
        month_path = os.path.join(selected_year_path, month_dir)
        
        # Check if it's a directory
        if not os.path.isdir(month_path):
            continue  # Skip if it's not a directory
        
        # Create a list of nc file paths within that month
        data_files = [f for f in os.listdir(month_path) if f.endswith('.nc')]
        
        # Initialize an empty list to store datasets
        datasets = []
        
        # Loop through each data file, open dataset, and append to datasets list
        for data_file in data_files:
            file_path = os.path.join(month_path, data_file)
            data = xr.open_dataset(file_path)
            datasets.append(data)
        
        # Concatenate datasets along time dimension
        combined_data = xr.concat(datasets, dim='time')
        
        # Calculate mean of 'sm_c4grass' along the time dimension (monthly average)
        mean_sm_c4grass_monthly = combined_data['sm_c4grass'].mean(dim='time')
        
        # Set spatial dimensions and CRS
        mean_sm_c4grass_monthly.rio.set_spatial_dims(x_dim='lon', y_dim='lat', inplace=True)
        mean_sm_c4grass_monthly.rio.write_crs('epsg:4326', inplace=True)
        
        # Clip data to Kenya boundary using Level 3 shapefile
        try:
            clipped_data_monthly = mean_sm_c4grass_monthly.rio.clip(kenya.geometry.apply(mapping), kenya.crs, drop=True)
            
            # Append monthly average to list for the selected year
            monthly_averages_selected_year.append(clipped_data_monthly)
            
            # Plotting monthly average
            fig, ax = plt.subplots(1, 1, figsize=(10, 10))
            kenya.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=0.5)
            clipped_data_monthly.plot(ax=ax, zorder=-1, cmap='viridis', cbar_kwargs={'label': 'Soil Moisture (sm_c4grass)'})
            plt.title(f'Average Soil Moisture ({month_dir} {selected_year})', fontsize=16)
            ax.set_xlabel('Longitude [degrees East]', fontsize=12)
            ax.set_ylabel('Latitude [degrees North]', fontsize=12)
            
            # Set axis ticks
            ax.set_xticks(range(34, 42, 2))
            ax.set_yticks(range(-4, 6, 2))           
            pdf.savefig(fig, bbox_inches='tight')
            plt.close(fig)
        except Exception as e:
            print(f"Error processing {month_dir} {selected_year}: {e}")

    # Concatenate monthly averages along time dimension to compute annual average
    if monthly_averages_selected_year:
        
        annual_average = xr.concat(monthly_averages_selected_year, dim='time').mean(dim='time')

        # Set spatial dimensions and CRS for annual average
        annual_average.rio.set_spatial_dims(x_dim='lon', y_dim='lat', inplace=True)
        annual_average.rio.write_crs('epsg:4326', inplace=True)
            
        # Clip data to Kenya boundary for annual average
        clipped_data_annual = annual_average.rio.clip(kenya.geometry.apply(mapping), kenya.crs, drop=True)
            
        # Plotting annual average
        fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        kenya.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=0.5)
        clipped_data_annual.plot(ax=ax, zorder=-1, cmap='viridis', cbar_kwargs={'label': 'Soil Moisture (sm_c4grass)'})
            
        plt.title(f'Annual Average Soil Moisture ({selected_year})', fontsize=16)
        ax.set_xlabel('Longitude [degrees East]', fontsize=12)
        ax.set_ylabel('Latitude [degrees North]', fontsize=12)
        ax.set_xticks(range(34, 42, 2))
        ax.set_yticks(range(-4, 6, 2))
            
        pdf.savefig(fig, bbox_inches='tight')
        plt.close(fig)
    else:
        print(f"Error processing annual average: {e}")

print(f"Plots saved to {pdf_path}")
